In [35]:
#https://austintexas.gov/docklessmobility

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [2]:
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
csv_file= "Dockless_Vehicle_Trips.csv"
dockless_data_df = pd.read_csv(csv_file)
dockless_data_df.head()

,ID,Device ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Origin Cell ID,Destination Cell ID,Year,Start Latitude,Start Longitude,End Latitude,End Longitude
0,000000cb-78a9-4b95-8ead-37c797abcf37,ad14e63b-940d-4ca3-84c6-0131147fba45,scooter,358.0,915.0,6/29/2018 17:30,6/29/2018 17:30,12/12/2018 21:36,6.0,17.0,5.0,5,9,13445,13916,2018.0,30.263,-97.762,30.259,-97.754
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,c5a230c9-7e80-4464-862f-f2b23f61d6d3,scooter,226.0,839.0,11/30/2018 11:00,11/30/2018 11:00,12/13/2018 15:43,11.0,11.0,5.0,9,9,14556,15186,2018.0,30.283,-97.745,30.284,-97.737
2,00002334-2f2b-41a0-9a04-8ff66be39f40,5a9f7815-bf0c-4fef-8bac-60efd4f23b10,scooter,324.0,1206.0,10/2/2018 15:15,10/2/2018 15:30,12/13/2018 15:49,10.0,15.0,2.0,9,9,15344,15032,2018.0,30.285,-97.734,30.291,-97.738
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,41fa0cf0-f433-4b3a-89f5-2c6cec6c367d,scooter,1096.0,0.0,9/2/2018 16:45,9/2/2018 17:00,12/13/2018 15:53,9.0,16.0,0.0,9,9,14707,14081,2018.0,30.268,-97.743,30.276,-97.752
4,00002c70-678b-4c09-a32c-da0266e329c6,ddd4bbc3-114e-4cbc-b078-860e3c28b11b,scooter,408.0,1144.0,9/28/2018 11:45,9/28/2018 11:45,12/12/2018 22:24,9.0,11.0,5.0,3,3,16114,15642,2018.0,30.242,-97.721,30.244,-97.729


In [4]:
new_dockless_data_df = dockless_data_df.rename(columns={'Trip Duration':'Trip_in_seconds', 
                                                        'Trip Distance':'Trip_in_meters'})



new_dockless_data_df.head()

,ID,Device ID,Vehicle Type,Trip_in_seconds,Trip_in_meters,Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Origin Cell ID,Destination Cell ID,Year,Start Latitude,Start Longitude,End Latitude,End Longitude
0,000000cb-78a9-4b95-8ead-37c797abcf37,ad14e63b-940d-4ca3-84c6-0131147fba45,scooter,358.0,915.0,6/29/2018 17:30,6/29/2018 17:30,12/12/2018 21:36,6.0,17.0,5.0,5,9,13445,13916,2018.0,30.263,-97.762,30.259,-97.754
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,c5a230c9-7e80-4464-862f-f2b23f61d6d3,scooter,226.0,839.0,11/30/2018 11:00,11/30/2018 11:00,12/13/2018 15:43,11.0,11.0,5.0,9,9,14556,15186,2018.0,30.283,-97.745,30.284,-97.737
2,00002334-2f2b-41a0-9a04-8ff66be39f40,5a9f7815-bf0c-4fef-8bac-60efd4f23b10,scooter,324.0,1206.0,10/2/2018 15:15,10/2/2018 15:30,12/13/2018 15:49,10.0,15.0,2.0,9,9,15344,15032,2018.0,30.285,-97.734,30.291,-97.738
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,41fa0cf0-f433-4b3a-89f5-2c6cec6c367d,scooter,1096.0,0.0,9/2/2018 16:45,9/2/2018 17:00,12/13/2018 15:53,9.0,16.0,0.0,9,9,14707,14081,2018.0,30.268,-97.743,30.276,-97.752
4,00002c70-678b-4c09-a32c-da0266e329c6,ddd4bbc3-114e-4cbc-b078-860e3c28b11b,scooter,408.0,1144.0,9/28/2018 11:45,9/28/2018 11:45,12/12/2018 22:24,9.0,11.0,5.0,3,3,16114,15642,2018.0,30.242,-97.721,30.244,-97.729


In [5]:

new_dockless_df = new_dockless_data_df[["ID", "Vehicle Type", "Trip_in_seconds", "Trip_in_meters", "Start Time" ]].copy()

new_dockless_df.head()





,ID,Vehicle Type,Trip_in_seconds,Trip_in_meters,Start Time
0,000000cb-78a9-4b95-8ead-37c797abcf37,scooter,358.0,915.0,6/29/2018 17:30
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,scooter,226.0,839.0,11/30/2018 11:00
2,00002334-2f2b-41a0-9a04-8ff66be39f40,scooter,324.0,1206.0,10/2/2018 15:15
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,scooter,1096.0,0.0,9/2/2018 16:45
4,00002c70-678b-4c09-a32c-da0266e329c6,scooter,408.0,1144.0,9/28/2018 11:45


In [12]:
new_dockless_data_d["Start Time"][:2].values

array(['6/29/2018 17:30', '11/30/2018 11:00'], dtype=object)

In [50]:
new_dockless_data_df = new_dockless_data_df.dropna(subset=["Start Time"])

In [53]:
def round_nearest_hour(input_time):
    try:
        time = datetime.strptime(input_time, "%m/%d/%Y %H:%M")
        time = time + timedelta(minutes=30)
        month = time.month
        year = time.year
        day = time.day
        hour = time.hour
        minute = 0
        return datetime(year, month, day, hour, minute)
    except:
        print("Error: ", input_time)
round_nearest_hour('6/29/2018 23:20')

datetime.datetime(2018, 6, 29, 23, 0)

In [54]:
new_dockless_data_df["Start Time (rounded)"] = new_dockless_data_df["Start Time"].apply(round_nearest_hour)
new_dockless_data_df.head()

,ID,Device ID,Vehicle Type,Trip_in_seconds,Trip_in_meters,Start Time,End Time,Modified Date,Month,Hour,...,Council District (Start),Council District (End),Origin Cell ID,Destination Cell ID,Year,Start Latitude,Start Longitude,End Latitude,End Longitude,Start Time (rounded)
0,000000cb-78a9-4b95-8ead-37c797abcf37,ad14e63b-940d-4ca3-84c6-0131147fba45,scooter,358.0,915.0,6/29/2018 17:30,6/29/2018 17:30,12/12/2018 21:36,6.0,17.0,...,5,9,13445,13916,2018.0,30.263,-97.762,30.259,-97.754,2018-06-29 18:00:00
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,c5a230c9-7e80-4464-862f-f2b23f61d6d3,scooter,226.0,839.0,11/30/2018 11:00,11/30/2018 11:00,12/13/2018 15:43,11.0,11.0,...,9,9,14556,15186,2018.0,30.283,-97.745,30.284,-97.737,2018-11-30 11:00:00
2,00002334-2f2b-41a0-9a04-8ff66be39f40,5a9f7815-bf0c-4fef-8bac-60efd4f23b10,scooter,324.0,1206.0,10/2/2018 15:15,10/2/2018 15:30,12/13/2018 15:49,10.0,15.0,...,9,9,15344,15032,2018.0,30.285,-97.734,30.291,-97.738,2018-10-02 15:00:00
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,41fa0cf0-f433-4b3a-89f5-2c6cec6c367d,scooter,1096.0,0.0,9/2/2018 16:45,9/2/2018 17:00,12/13/2018 15:53,9.0,16.0,...,9,9,14707,14081,2018.0,30.268,-97.743,30.276,-97.752,2018-09-02 17:00:00
4,00002c70-678b-4c09-a32c-da0266e329c6,ddd4bbc3-114e-4cbc-b078-860e3c28b11b,scooter,408.0,1144.0,9/28/2018 11:45,9/28/2018 11:45,12/12/2018 22:24,9.0,11.0,...,3,3,16114,15642,2018.0,30.242,-97.721,30.244,-97.729,2018-09-28 12:00:00


In [55]:
new_dockless_df = new_dockless_data_df[["ID", "Vehicle Type", "Trip_in_seconds", "Trip_in_meters", "Start Time (rounded)" ]].copy()

#new_dockless_df.head()
new_dockless_df

,ID,Vehicle Type,Trip_in_seconds,Trip_in_meters,Start Time (rounded)
0,000000cb-78a9-4b95-8ead-37c797abcf37,scooter,358.0,915.0,2018-06-29 18:00:00
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,scooter,226.0,839.0,2018-11-30 11:00:00
2,00002334-2f2b-41a0-9a04-8ff66be39f40,scooter,324.0,1206.0,2018-10-02 15:00:00
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,scooter,1096.0,0.0,2018-09-02 17:00:00
4,00002c70-678b-4c09-a32c-da0266e329c6,scooter,408.0,1144.0,2018-09-28 12:00:00
5,00002f2a-0bcd-4446-bb39-df9fd62be813,scooter,1094.0,2631.0,2018-07-22 17:00:00
6,00003324-85b7-45d0-ada2-31a5db5560fe,scooter,705.0,1248.0,2018-10-28 10:00:00
7,00003584-6e34-4ccc-9968-b1ae3f47b1d7,scooter,143.0,702.0,2018-10-08 18:00:00
8,0000485a-1622-4fed-8fbd-f255552a882e,scooter,215.0,180.0,2018-06-16 22:00:00
9,00004fe9-fdb2-4b55-ab44-789beba2d1d9,scooter,197.0,530.0,2018-10-25 11:00:00


In [56]:
#import pymongo

In [57]:
#conn = 'mongodb://localhost:27017'
#client = pymongo.MongoClient(conn)

In [ ]:
#db = client.dockless_vehicle_db
#collection = db.dockless